In [1]:
import pandas as pd
import sys

PATH = '/Users/danil/Documents/github/sweet_RS/'
sys.path.append(str(PATH))

from src.utils import read_pickle

In [2]:
sbmt = pd.read_csv('../data/raw/submission.csv')
print(sbmt.shape)
sbmt.head()

(1695, 2)


,user_id,movie_id
0,16975709365084587137,0 0 0 0 0
1,10225505309438393880,0 0 0 0 0
2,337894458057354820,0 0 0 0 0
3,16389753737490268206,0 0 0 0 0
4,12377183607123393787,0 0 0 0 0


# top 5 pred

In [30]:
most_pop = [12304, 7735, 15285, 17518, 19026]

In [18]:
predict = sbmt.copy()
predict = predict.drop(' movie_id', 1)

for ind, user in enumerate(sbmt.iterrows()):
    #print(user[1]['user_id'])
    
    pred = ' '.join([str(i) for i in most_pop])
    
    predict.loc[ind, 'movie_id'] = pred
    
predict.to_csv('../data/submits/top_5.csv', index = False)

# 5 from top 100

In [23]:
import pickle

with open('../data/processed/random_5_from_top100.pickle', 'rb') as f:
    preds = pickle.load(f)

In [25]:
predict = sbmt.copy()
predict = predict.drop(' movie_id', 1)

for ind, user in enumerate(sbmt.iterrows()):
    #print(user[1]['user_id'])
    
    pred = ' '.join([str(i) for i in preds[user[1]['user_id']]])
    
    predict.loc[ind, 'movie_id'] = pred
    
predict.to_csv('../data/submits/random_5_from_top_100.csv', index = False)

# lightFM baseline

In [53]:
#preds = read_pickle('../data/processed/light_fm.pickle')
preds = read_pickle('../data/processed/light_fm_new_idbm.pickle')

predict = sbmt.copy()
predict = predict.drop(' movie_id', 1)

for ind, user in enumerate(sbmt.iterrows()):
    #print(user[1]['user_id'])
    
    pred = ' '.join([str(i) for i in preds[user[1]['user_id']]])
    
    predict.loc[ind, 'movie_id'] = pred
    
predict.to_csv('../data/submits/light_fm_new_idbm.csv', index = False)

LB 0.007

# lightFM tuned

In [29]:
preds = read_pickle('../data/processed/light_fm_tuned.pickle')

predict = sbmt.copy()
predict = predict.drop(' movie_id', 1)

for ind, user in enumerate(sbmt.iterrows()):
    #print(user[1]['user_id'])
    
    pred = ' '.join([str(i) for i in preds[user[1]['user_id']]])
    
    predict.loc[ind, 'movie_id'] = pred
    
predict.to_csv('../data/submits/light_fm_tuned.csv', index = False)

LB 0.006

# fav serials + lightFM

prepare top 5 serials

In [10]:
user_hist_df = pd.read_csv('../data/raw/movies_dataset_10 months.csv')
user_hist_df['ts'] = pd.to_datetime(user_hist_df['ts'])
user_hist_df['month'] = user_hist_df['ts'].dt.month

fav_serials = user_hist_df[(user_hist_df['episode_id'] != 0) & 
                           (user_hist_df['month'] == 7)].groupby(['user_id', 'movie_id'])['movie_id'].count()
fav_serials = fav_serials[fav_serials>=2].reset_index(name='cnt')

g = fav_serials.groupby(["user_id"]).apply(lambda x: x.sort_values(["cnt"], ascending = False)).reset_index(drop=True)

user_top_5_serials = g.groupby(["user_id"])['movie_id'].apply(lambda x: list(x[:5])).to_dict()


load best lightFM

In [31]:
preds = read_pickle('../data/processed/light_fm.pickle')

In [55]:
preds = read_pickle('../data/processed/light_fm_new_idbm.pickle')

In [58]:
predict = sbmt.copy()
predict = predict.drop(' movie_id', 1)

for ind, user in enumerate(sbmt.iterrows()):
    
    user_id = user[1]['user_id']
    #print(user[1]['user_id'])
    
    
    if user_top_5_serials.get(user_id):
    
        pref_serials = user_top_5_serials[user_id][:]
        num_of_serials = len(pref_serials)

        if num_of_serials < 5:
            additional = 5 - num_of_serials
            light_fm_pred = preds[user_id][:additional]

            pred = pref_serials + light_fm_pred
            
        else:
            pred = pref_serials
    else:
         pred = preds[user_id]
            
    pred = ' '.join([str(i) for i in pred])
    
    predict.loc[ind, 'movie_id'] = pred
    
predict.to_csv('../data/submits/fav_serials_light_fm_filtered_films_20.csv', index = False)

LB 0.01339

# covis

In [20]:
from operator import itemgetter 

In [37]:
user_hist_df_covis = user_hist_df[['user_id', 'movie_id']].drop_duplicates()

In [38]:
user_hist = user_hist_df_covis.groupby('user_id')['movie_id'].apply(set).to_dict()

In [39]:
covis = {}

for user in user_hist:
    curr_user_hist = user_hist[user]
    for film in curr_user_hist:
        if covis.get(film):
            for other in curr_user_hist:
                if other == film:
                    continue
                else:
                    if covis[film].get(other):
                        covis[film][other] += 1
                    else:
                        covis[film][other] = 1
        else:
            covis[film] = {}
            for other in curr_user_hist:
                if other == film:
                    continue
                else:
                    if covis[film].get(other):
                        covis[film][other] += 1
                    else:
                        covis[film][other] = 1
            

In [40]:
covis_old = covis.copy()

In [41]:
for film in covis:
    covis[film] = dict(sorted(covis[film].items(), key = itemgetter(1), reverse = True)[:30]) 
    total = sum(covis[film].values())
    for f in covis[film]:
        covis[film][f]/=total

In [42]:
def pred_for_user(covis, user_hist, user, N):
    recoms = {}
    for film in user_hist[user]:
        cov = covis[film]
        for rec in cov:
            if recoms.get(rec):
                recoms[rec] += covis[film][rec]
            else:
                recoms[rec] = covis[film][rec]
    N_2 = N + 50
    res = dict(sorted(recoms.items(), key = itemgetter(1), reverse = True)[:N_2]) 
    
    res_f = list(set(res.keys()).difference(user_hist[user]))[:N]
    return res_f

In [43]:
predict = sbmt.copy()
predict = predict.drop(' movie_id', 1)

for ind, user in enumerate(sbmt.iterrows()):
    #print(user[1]['user_id'])
    
    pred = pred_for_user(covis, user_hist, user[1]['user_id'], 5)
    if len(pred)<5:
        addit = 5 - len(pred)
        pred += most_pop[:addit]
        print(user)
    pred = ' '.join([str(i) for i in pred])
    
    predict.loc[ind, 'movie_id'] = pred
    
predict.to_csv('../data/submits/covis.csv', index = False)

(1264, user_id      11254796898083103047
 movie_id               0 0 0 0 0
Name: 1264, dtype: object)


LB 0.0031

# fav + covis

In [ ]:
predict = sbmt.copy()
predict = predict.drop(' movie_id', 1)

for ind, user in enumerate(sbmt.iterrows()):
    
    user_id = user[1]['user_id']
    #print(user[1]['user_id'])
    
    
    if user_top_5_serials.get(user_id):
    
        pref_serials = user_top_5_serials[user_id]
        num_of_serials = len(pref_serials)

        if num_of_serials < 5:
            additional = 5 - num_of_serials
            covis_pred = pred_for_user(covis, user_hist, user_id, additional)

            pred = pref_serials + covis_pred
            
        else:
            pred = pref_serials
    else:
         pred = pred_for_user(covis, user_hist, user_id, 5)
            
    pred = ' '.join([str(i) for i in pred])
    
    predict.loc[ind, 'movie_id'] = pred

In [ ]:
predict.to_csv('../data/submits/fav_serial_covis.csv', index = False)

# ALS baseline

In [66]:
preds = read_pickle('../data/processed/als_2.pickle')

predict = sbmt.copy()
predict = predict.drop(' movie_id', 1)

for ind, user in enumerate(sbmt.iterrows()):
    #print(user[1]['user_id'])
    
    pred = ' '.join([str(i) for i in preds[user[1]['user_id']]])
    
    predict.loc[ind, 'movie_id'] = pred
    
predict.to_csv('../data/submits/als_2.csv', index = False)

# fav serials + ALS

prepare top 5 serials

In [3]:
user_hist_df = pd.read_csv('../data/raw/movies_dataset_10 months.csv')
user_hist_df['ts'] = pd.to_datetime(user_hist_df['ts'])
user_hist_df['month'] = user_hist_df['ts'].dt.month

fav_serials = user_hist_df[(user_hist_df['episode_id'] != 0) & 
                           (user_hist_df['month'] == 7)].groupby(['user_id', 'movie_id'])['movie_id'].count()
fav_serials = fav_serials[fav_serials>=3].reset_index(name='cnt')

g = fav_serials.groupby(["user_id"]).apply(lambda x: x.sort_values(["cnt"], ascending = False)).reset_index(drop=True)

user_top_5_serials = g.groupby(["user_id"])['movie_id'].apply(lambda x: list(x[:5])).to_dict()


In [71]:
preds = read_pickle('../data/processed/als_2.pickle')

In [72]:
predict = sbmt.copy()
predict = predict.drop(' movie_id', 1)

for ind, user in enumerate(sbmt.iterrows()):
    
    user_id = user[1]['user_id']
    #print(user[1]['user_id'])
    
    
    if user_top_5_serials.get(user_id):
    
        pref_serials = user_top_5_serials[user_id][:]
        num_of_serials = len(pref_serials)

        if num_of_serials < 5:
            additional = 5 - num_of_serials
            light_fm_pred = preds[user_id][:additional]

            pred = pref_serials + light_fm_pred
            
        else:
            pred = pref_serials
    else:
         pred = preds[user_id]
            
    pred = ' '.join([str(i) for i in pred])
    
    predict.loc[ind, 'movie_id'] = pred
    
predict.to_csv('../data/submits/fav_serials_3_als_2.csv', index = False)

# LMF baseline

In [65]:
preds = read_pickle('../data/processed/lmf.pickle')

predict = sbmt.copy()
predict = predict.drop(' movie_id', 1)

for ind, user in enumerate(sbmt.iterrows()):
    #print(user[1]['user_id'])
    
    pred = ' '.join([str(i) for i in preds[user[1]['user_id']]])
    
    predict.loc[ind, 'movie_id'] = pred
    
predict.to_csv('../data/submits/lmf.csv', index = False)

# fav serial + item simil

In [89]:
preds = read_pickle('../data/processed/item_simil_10.pickle')

In [92]:
predict = sbmt.copy()
predict = predict.drop(' movie_id', 1)

for ind, user in enumerate(sbmt.iterrows()):
    
    user_id = user[1]['user_id']
    #print(user[1]['user_id'])
    
    
    if user_top_5_serials.get(user_id):
    
        pref_serials = user_top_5_serials[user_id][:]
        num_of_serials = len(pref_serials)

        if num_of_serials < 5:
            additional = 5 - num_of_serials
            light_fm_pred = preds[user_id][:additional]

            pred = pref_serials + light_fm_pred
            
        else:
            pred = pref_serials
    else:
         pred = preds[user_id][:5]
            
    pred = ' '.join([str(i) for i in pred])
    
    predict.loc[ind, 'movie_id'] = pred
    
predict.to_csv('../data/submits/fav_serials_item_simil.csv', index = False)

In [93]:
predict

,user_id,movie_id
0,16975709365084587137,17729 18262 18611 18507 18964
1,10225505309438393880,17079 13122 17850 3506 13950
2,337894458057354820,16676 16830 17186 16829 17999
3,16389753737490268206,17909 19954 19279 18548 18423
4,12377183607123393787,18652 18308 18520 18153 18413
...,...,...
1690,16073255054457636768,16757 18534 7351 17966 13378
1691,18217982949448186449,18534 17966 7347 17756 13378
1692,8493046014808115881,18652 18413 18520 9311 18807
1693,6524731807859816855,18520 18413 18652 9311 18153


# common in top 10 from als_2, item2item, light_fm

In [75]:
#preds = read_pickle('../data/processed/light_fm.pickle')
als = read_pickle('../data/processed/als_2_10.pickle')
ite2item = read_pickle('../data/processed/item_simil_10.pickle')
light_fm = read_pickle('../data/processed/light_fm_new_idbm_10.pickle')

In [78]:
from collections import Counter

In [82]:
[x[0] for x in pred if x[1] >= 2]

[7347, 7351, 18080]

In [87]:
predict = sbmt.copy()
predict = predict.drop(' movie_id', 1)

for ind, user in enumerate(sbmt.iterrows()):
    user = user[1]['user_id']
    
    #pred = Counter(als[user] + ite2item[user] + light_fm[user]).most_common(5)
    pred = Counter(als[user] + light_fm[user]).most_common(5)
    pred = [x[0] for x in pred if x[1] >= 2]
    
    if len(pred) < 5:
        additional = 5 - len(pred)
        light_fm_pred = light_fm[user][:additional]

        pred += light_fm_pred
    
    pred = ' '.join([str(i) for i in pred])
    
    predict.loc[ind, 'movie_id'] = pred
    
predict.to_csv('../data/submits/ensemble_als_lightfm.csv', index = False)

# 2_lvl baseline

In [4]:
import pickle

with open('../data/processed/random_5_from_top100.pickle', 'rb') as f:
    random_preds = pickle.load(f)

In [7]:
#preds = read_pickle('../data/processed/2_lvl_30000.pickle')
preds = read_pickle('../data/processed/2_lvl_xgbm.pickle')

predict = sbmt.copy()
predict = predict.drop(' movie_id', 1)

for ind, user in enumerate(sbmt.iterrows()):
    #print(user[1]['user_id'])
    user = user[1]['user_id']
    
    if preds.get(user):
        #pred = ' '.join([str(i) for i in preds[user]])
        pred = preds[user]
    elif user_top_5_serials.get(user):
        print('top_5')
        if len(user_top_5_serials[user]) < 5:
            additional = len(user_top_5_serials[user]) - 5
            addit = random_preds[user][:additional]
            
            pred = user_top_5_serials[user] + addit
    else:
        print('random')
        pred = random_preds[user][:5]

    pred = ' '.join([str(i) for i in pred])
    
    predict.loc[ind, 'movie_id'] = pred
    
predict.to_csv('../data/submits/2_lvl_xgbm.csv', index = False)

random
random
random
random
random
random
random
random
random
random
random
random
random
random
random


In [8]:
predict

,user_id,movie_id
0,16975709365084587137,19252 16693 16032 10447 17550
1,10225505309438393880,19252 2973 10337 2164 2165
2,337894458057354820,17261 10920 9335 19207 16032
3,16389753737490268206,17261 16693 2973 10920 19252
4,12377183607123393787,10920 19383 19207 17041 9408
...,...,...
1690,16073255054457636768,19353 15679 16601 17261 5
1691,18217982949448186449,10920 5 19207 18576 9256
1692,8493046014808115881,17261 10920 9335 9221 2973
1693,6524731807859816855,17261 9335 10920 19207 17041
